In [1]:
from Crypto.Random.random import randrange
from Crypto.Hash import SHA256
import asn1

In [2]:
p = 57896044630612021680684936114742422271145183870487080309667128995208157569947
q = 28948022315306010840342468057371211135571302038761442251594012761075345324491
a = 1
b = 19750513962881385028059495396984460236743646692126413053976069443380491067343
xP = 43490682822985073571091311123441225129011272278165566160439297012894969619553
yP = 53273700124912449490307054424387372532482586733448415163119878489682918137700
E = EllipticCurve(GF(p), [a, b])
P = E(xP, yP)
d = 8784854584545465456546545403424680573712111355713020387614422515975345324491
Q = d * P

In [3]:
def Encode(Q, r, s):
    xq = int(Q[0])
    yq = int(Q[1])
    xp = int(P[0])
    yp = int(P[1])

    encoder = asn1.Encoder()
    encoder.start()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.enter(asn1.Numbers.Set)
    encoder.enter(asn1.Numbers.Sequence)
    encoder.write(b'\x80\x06\x07\x00', asn1.Numbers.OctetString)
    encoder.write(b'gostSignKey', asn1.Numbers.UTF8String)
    encoder.enter(asn1.Numbers.Sequence)
    encoder.write(xq, asn1.Numbers.Integer)
    encoder.write(yq, asn1.Numbers.Integer)
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.enter(asn1.Numbers.Sequence)
    encoder.write(p, asn1.Numbers.Integer)
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.write(a, asn1.Numbers.Integer)
    encoder.write(b, asn1.Numbers.Integer)
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence)  
    encoder.write(xp, asn1.Numbers.Integer)
    encoder.write(yp, asn1.Numbers.Integer)
    encoder.leave()
    encoder.write(q, asn1.Numbers.Integer)
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence) 
    encoder.write(r, asn1.Numbers.Integer)
    encoder.write(s, asn1.Numbers.Integer)
    encoder.leave()
    encoder.leave()
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.leave()
    encoder.leave()

    return encoder.output()

def Decode(decoder, params):
    while not decoder.eof():
        try:
            tag = decoder.peek()
            if tag.nr == asn1.Numbers.Null:
                break
            if tag.typ == asn1.Types.Primitive:
                tag, value = decoder.read()
                if tag.nr == asn1.Numbers.Integer:
                    params.append(value)
            else:
                decoder.enter()
                params = Decode(decoder, params)
                decoder.leave()
        except asn1.Error:
            break

    return params  

def sign(filename):
    fin = open(filename, 'rb')
    text = fin.read()
    fin.close()
    
    sha = SHA256.new()
    sha.update(text)
    h = int.from_bytes(sha.digest(), 'big')
    
    e = h % q
    if e == 0:
        e = 1
    
    r = 0
    while r == 0:
        k = randrange(int(1), int(q - 1))
        C = k * P
        r = int(C[0]) % q
        
    s = (r * d + k * e) % q
    
    asn1sign = Encode(Q, r, s)

    fout = open('{}_signature.asn1'.format(filename), 'wb')
    fout.write(asn1sign)
    fout.close()
    
def verify(filename, signfilename):
    fin1 = open(filename, 'rb')
    text = fin1.read()
    fin1.close()
    
    fin2 = open(signfilename, 'rb')
    sign = fin2.read()
    fin2.close()
    
    decoder = asn1.Decoder()
    decoder.start(sign)
    params = []
    params = Decode(decoder, params)
    
    r = params[8]
    s = params[9]
    if r >= q or s >= q:
        print('Signature is not correct')
        return
    
    sha = SHA256.new()
    sha.update(text)
    h = int.from_bytes(sha.digest(), 'big')
    
    e = h % q
    if e == 0:
        e = 1
        
    v = pow(e, -1, q)
    z1 = (s * v) % q
    z2 = -(r * v) % q
    
    C = int(z1) * P + int(z2) * Q
    R = int(C[0]) % q
    
    if r == int(R):
        print('Signature is correct')
    else:
        print('Signature is not correct')

In [6]:
sign(f)

In [11]:
f = '2.jpg'
verify(f, '1.png_signature.asn1')

Signature is not correct
